In [65]:
import numpy as np
import struct
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import scale
import cv2 as cv
from math import floor, ceil

In [66]:
with open('../scenes/sample.dat', 'rb') as f:
    fileContent = f.read()
    width, height, spp, SL = struct.unpack("LLLL", fileContent[:32])
    samples = struct.unpack("f" * ((len(fileContent) - 32) // 4), fileContent[32:])
    raw = np.array(samples)
    samples = np.reshape(raw, (width, height, spp, SL))
print(width, height, spp, SL)
print(samples[10,5,4,:])    

100 100 8 28
[ 5.82861328e+00  1.05459538e+01  2.56491732e-02  2.56491732e-02
  4.10386808e-02 -4.00000000e+02  2.95105316e+02 -1.33042297e+01
  1.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e-01
  5.00000000e-01  8.00000012e-01  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  1.25136793e-01
  3.10504138e-01  5.03994346e-01  0.00000000e+00  0.00000000e+00]


In [67]:
def preprocess(x, y, b, M):
    # Sample neighbors based on gaussian distribution
    indexN = np.random.multivariate_normal(mean=[x, y], cov=(b/4) * np.identity(2), size=M-spp)
    indexN = np.clip(indexN, 0, 99)
    indexN = np.around(indexN)
    randSampleInPixel = np.random.randint(7, size=(M-spp, 1))
    indexN = np.concatenate((indexN,randSampleInPixel), axis = 1)
    sampleMapper = lambda v: samples[int(v[0]), int(v[1]), int(v[2]),:]
    N = np.apply_along_axis(sampleMapper, 1, indexN)
    N = np.concatenate((samples[x, y, :, :], N), axis = 0)
    # Cluster samples
    discard = []
    featOffset = 2
    for i in range(4):
        feat = N[:, featOffset:featOffset + 3]
        featOffset += 3
        m = np.mean(feat)
        std = np.std(feat)
        if std > 0.1:
            for j in range(M):
                if featOffset == 5:
                    if np.abs(m - np.mean(feat[j])) > 30 * std:
                        discard.append(j)
                else:
                    if np.abs(m - np.mean(feat[j])) > 3 * std:
                        discard.append(j)
    newN = []
    for i in range(M):
        if i not in discard:
            newN.append(N[i])
    # Standardize neighbors
    N = scale(N)
    return N

In [68]:
def sumD(Dxx):
    return Dxx[0] + Dxx[1] + Dxx[2]

def getRange(x):
    return np.arange(floor(x.min()), ceil(x.max()), 1, dtype=int)

def mutual_info(x, y):
    cc_xy = np.histogram2d(x, y)[0]
    mi = mutual_info_score(None, None, contingency=cc_xy)
    return mi

def computeFeatureWeights(t, N):
    # cn = {N[:,2], N[:,3] ,N[:,4]} # R, G, B
    cn = [2, 3, 4] # R, G, B
    # rn = {N[:,0], N[:,1], N[:,23], N[:,24]} # X, Y, U, V
    rn = [0, 1, 23, 24] # X, Y, U, V
    # pn = {N[:,0], N[:,1]} # X, Y
    pn = [0, 1] # X, Y

    FEATURECOUNT = 12
    # f = N[:,2:14]
    f = list(range(2, 14))
    
    Dcr = [0.0, 0.0, 0.0] # Deps between color and random
    Dcp = [0.0, 0.0, 0,0] # Deps between color and position
    Dcf = [0.0, 0.0, 0.0] # Deps between color and feature
    Wcr = [0.0, 0.0, 0.0] # 
    epsilon = 1e-10
    alpha = [0.0, 0.0, 0.0]
    
    Dfc = np.zeros(FEATURECOUNT)
    for k in range(3):
        for j in range(4):
            Dcr[k] += mutual_info(N[:,cn[k]], N[:,rn[j]])
        for l in range(2):
            Dcp[k] += mutual_info(N[:,cn[k]], N[:,pn[l]])
        for l in range(12):
            tmp = mutual_info(N[:,cn[k]],N[:,f[l]])
            Dcf[k] += tmp
            Dfc[l] += tmp

        Wcr[k] = Dcr[k] / (Dcr[k] + Dcp[k] + epsilon)
        alpha[k] = max(1 - (1 + 0.1 * t) * Wcr[k], 0)
        
    Dfr = np.zeros(FEATURECOUNT)
    Dfp = np.zeros(FEATURECOUNT)
    Wfr = np.zeros(FEATURECOUNT)
    Wfc = np.zeros(FEATURECOUNT)
    beta = np.zeros(FEATURECOUNT)
    
    for k in range(FEATURECOUNT):
        for l in range(4):
            Dfr[k] += mutual_info(N[:,f[k]], N[:,rn[l]])
        for l in range(2):
            Dfp[k] += mutual_info(N[:,f[k]], N[:,pn[l]])
        
        Wfr[k] = Dfr[k] / (Dfr[k] + Dfp[k] + epsilon)

        Wfc[k] = Dfc[k] / (sumD(Dcr) + sumD(Dcp) + sumD(Dcf))
        beta[k] = max(1 - (1 + 0.1 * t) * Wfr[k], 0)
        
    return alpha, beta, Wcr



In [69]:
def filterColor(x, y, N, alpha, beta, Wcr):
    P = samples[x, y, : , :]
    v = 0.002
    vc = v / (1 - Wcr) ** 2
    vf = vc
    for i in P:
        newColor = 0
        w = 0
        for j in N:
            # TODO: Gaussian formula
            wij = np.exp(-0.5 * (1 / vc)) * np.exp(-0.5 * (1 / vf))
            newColor += wij * cj
            w += wij
        newColor /= w
        # TODO: Update color in samples

In [70]:
def boxFilter():
    colors = np.zeros([width, height, 3])
    colors[:,:,2] = samples[:,:,0:8,2].mean(-1)
    colors[:,:,1] = samples[:,:,0:8,3].mean(-1)
    colors[:,:,0] = samples[:,:,0:8,4].mean(-1)

    cv.imwrite('testres/color.jpg', colors * 255.0)

In [71]:
boxSizes = [10, 7, 5, 3]
for t in range(len(boxSizes)):
    b = boxSizes[t]
    maxNumOfSamples = (b * b * spp) // 2
    for i in range(height):
        for j in range(width):
            N = preprocess(i, j, b, maxNumOfSamples)
            alpha, beta, Wcr = computeFeatureWeights(t, N)
            print(alpha, beta, Wcr)
            break
        break
            # TODO: add filterColor()

[0.2975169690328686, 0.2975169690328686, 0.2939777652217862] [0.29751697 0.29751697 0.29397777 0.26636734 0.23434784 0.23998529
 1.         1.         1.         1.         1.         1.        ] [0.7024830309671314, 0.7024830309671314, 0.7060222347782138]
[0.21033318652775546, 0.21033318652775546, 0.2172147557058175] [0.21033319 0.21033319 0.21721476 0.19698649 0.16314124 0.17107217
 1.         1.         1.         1.         1.         1.        ] [0.717878921338404, 0.717878921338404, 0.7116229493583477]
[0.16716511983431137, 0.16716511983431137, 0.17594303753555673] [0.16716512 0.16716512 0.17594304 0.20128818 0.13654163 0.13920736
 0.99999267 0.99999267 0.99999267 0.99999267 0.99999267 0.99999267] [0.6940290668047405, 0.6940290668047405, 0.6867141353870361]
[0.1476958840542335, 0.1476958840542335, 0.15731172562343432] [0.14769588 0.14769588 0.15731173 0.13788486 0.12657818 0.12208259
 1.         1.         1.         1.         1.         1.        ] [0.6556185507275126, 0.655618